<a href="https://colab.research.google.com/github/WinetraubLab/coregister-xy/blob/main/coregister_xy_cell_to_cell.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>
<a href="https://github.com/WinetraubLab/coregister-xy/blob/main/coregister_xy_cell_to_cell.ipynb" target="_blank">
  <img src="https://img.shields.io/badge/view%20in-GitHub-blue" alt="View in GitHub"/>
</a>

# Overview
Cell-to-cell alignment of a single 2D fluorescent/histology image to 3D OCT volume.

In [ ]:
# @title Inputs

# Image paths
oct_volume_image_path = ""      # Path to OCT volume
hist_image_path = ""            # Path to fluorescent image

# For rough alignment
anchor_points_slide_location_pix = []        # Coordinates in px from fluorescent/histology slide image
anchor_points_physical_location_mm = []      # Matching coordinates in mm from photobleach template
smoothing = 0.5                              # 0 for fully elastic, 1 for fully affine
oct_scan_x_range_mm = [0, 1]                 # OCT scan range in x, from photobleach template
oct_scan_y_range_mm = [0, 1]                 # OCT scan range in y, from photobleach template

# Registration coarseness
registration_algorithm = 'rough and cell-level'      # Use 'rough' or 'rough and cell-level'

# For segmentation
z_buffer_um = 20                # Desired amount of extra space above/below the rough fit estimated from the above provided coordinate pairs.
barcode_focus_oct_slice = 120   # Voxel depth/slice number in OCT volume where photobleach barcode z=0.
oct_xy_downscale_factor = 2     # Optional: Downsampling to increase segmentation speed
avg_cell_diameter_px = 10       # Expected average diameter of the cells to segment, in the raw volume (no downsampling)
hist_valid_cell_segmentation_path = None # Optional: binary mask for histology image to denote areas to keep/discard cell segmentations

# For point matching (to identify pairs of points to consider as potential true correspondences)
initial_pairs_dist_threshold_um = 40 # Approximate accuracy of rough alignment, in um
z_weight = 1.5                       # Increase this value the more confident you are in your z depth estimations from barcodes.
                                     # Recommend keeping this value between 1 to 5.

In [1]:
# @title Environment Setup

!pip install cellpose==2.2 --quiet
!git clone https://github.com/WinetraubLab/coregister-xy.git
%cd coregister-xy

from cell_to_cell_registration import segment_cells
from cell_to_cell_registration import PCR99a
from plane.fit_plane_elastic import FitPlaneElastic

from scipy.interpolate import RegularGridInterpolator
from scipy.spatial import cKDTree
from skimage.measure import regionprops, label
from cellpose import io, models, plot

import matplotlib.pyplot as plt
import cv2
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.4/170.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 36.5 MB/s eta 0:00:00
Cloning into 'coregister-xy'...
remote: Enumerating objects: 1725, done.
remote: Counting objects: 100% (464/464), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 1725 (delta 314), reused 234 (delta 234), pack-reused 1261 (from 2)
Receiving objects: 100% (1725/1725), 25.85 MiB | 25.58 MiB/s, done.
Resolving deltas: 100% (1079/1079), done.
/content/coregister-xy


## Rough Alignment

In [ ]:
hist_img = io.imread(hist_image_path)
assert hist_img is not None, f"Failed to load image: {hist_image_path}"

fp = FitPlaneElastic.from_points(anchor_points_slide_location_pix, anchor_points_physical_location_mm, smoothing=smoothing, consistency_check=False)
rough_align_2d_image = fp.image_to_physical_z_projection(hist_img, oct_scan_x_range_mm, oct_scan_y_range_mm, 1e-3)
surface_z_um = fp.get_image_surface_z(oct_scan_x_range_mm, oct_scan_y_range_mm, 1e-3) * 1000

plt.imshow(surface_z_um)
plt.colorbar()
plt.title("Surface Depth Estimated from Photobleach Barcodes")
plt.show()

if "cell" in registration_algorithm:
    do_cell_to_cell_alignment = True

if not do_cell_to_cell_alignment:
    sampled_oct_image = np.zeros_like(rough_align_2d_image) # Will be sampled using surface_z_um

## Cell-to-Cell Alignment

In [ ]:
# @title Segmentation
if do_cell_to_cell_alignment:

    # Placeholder
    oct_centroids_xyz = np.random.rand(20, 3)
    hist_centroids_xyz = np.random.rand(15, 3)

    # Visualization of cell segmentations


In [ ]:
# @title Matching Initial Pairs
if do_cell_to_cell_alignment:

    # Placeholder
    matched_pairs = np.hstack([oct_centroids_xyz[:5], hist_centroids_xyz[:5]]).T

In [ ]:
# @title Compute Transform
if do_cell_to_cell_alignment:

    # Placeholder
    T = np.eye(4)


In [ ]:
# @title Sample OCT on Plane
if do_cell_to_cell_alignment:

    # Placeholder
    sampled_oct_image = np.zeros_like(rough_align_2d_image)

    # Visualization of tilted plane
